In [1]:
from datasets import load_dataset, DatasetDict

timit = DatasetDict()

timit["train"] = load_dataset("timit_asr", data_dir="timit", split="train")
timit["test"] = load_dataset("timit_asr", data_dir="timit", split="test")

timit

/Users/henrytsyu/imperial/model/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [2]:
model_name = "openai/whisper-tiny.en"

In [3]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)

In [17]:
from transformers import WhisperTokenizerFast

tokenizer = WhisperTokenizerFast(
    tokenizer_file="tokenizer.json",
)
tokenizer.pad_token = tokenizer.eos_token

In [18]:
# Ensure sampling rate is 16kHz
timit["train"][0]

{'input_features': [[-0.5182292461395264,
   -0.4007164239883423,
   -0.46678662300109863,
   -0.42608416080474854,
   -0.47227048873901367,
   -0.40634047985076904,
   -0.35498344898223877,
   -0.4913790225982666,
   -0.3808702230453491,
   -0.4441796541213989,
   -0.392414927482605,
   -0.41258931159973145,
   -0.45893287658691406,
   -0.364803671836853,
   -0.3843446969985962,
   -0.3947383165359497,
   -0.3726891279220581,
   -0.44801008701324463,
   -0.4330437183380127,
   -0.5322024822235107,
   -0.5761305093765259,
   -0.5585017204284668,
   -0.7242892980575562,
   -0.6578612327575684,
   -0.49004673957824707,
   -0.7142006158828735,
   -0.6691831350326538,
   -0.44230926036834717,
   -0.48997974395751953,
   -0.6836991310119629,
   -0.38622212409973145,
   -0.5285007953643799,
   -0.5061051845550537,
   -0.5988056659698486,
   -0.5745967626571655,
   -0.5991696119308472,
   -0.6280757188796997,
   -0.5039852857589722,
   -0.4469858407974243,
   -0.6574121713638306,
   -0.786575

In [19]:
def prepare_dataset(example):
    # load audio data as 16kHz
    audio = example["audio"]

    # compute log-Mel input features from input audio array
    example["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    example["labels"] = tokenizer(" ".join(example["phonetic_detail"]["utterance"])).input_ids
    return example

In [ ]:
timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=1)

In [21]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name)

In [22]:
timit

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1680
    })
})

In [23]:
import torch

from dataclasses import dataclass
from typing import Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    feature_extractor: WhisperFeatureExtractor
    tokenizer: WhisperTokenizerFast
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [24]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [25]:
import evaluate

metric = evaluate.load("wer")

In [26]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [27]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=f"./{model_name}-timit",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

/Users/henrytsyu/imperial/model/venv/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
)

/var/folders/cb/plxh87nd78gd3fr6_gzk2ktr0000gn/T/ipykernel_1590/3856086430.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()